In [16]:
import glob
import json
import os

import librosa
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import soundfile as sf 

import utils
import plots_improved as plots

# listen to audios
import IPython.display as ipd

In [17]:
# Parse the CSV data
df = pd.read_json("../data/input_data/muchomusic_musiccaps.json")

def is_single_sounding_event(prompt, answers):
    prompt_lower = prompt.lower()

    # Questions that typically require longer-term understanding
    long_term_keywords = [
        'genre', 'mood', 'atmosphere', 'tempo', 'style', 'overall', 'primary', 'mainly',
        'primarily', 'convey', 'purpose', 'suitable for', 'intended for', 'typically heard',
        'likely performed', 'cultural', 'influence', 'era', 'origin', 'recording quality',
        'emotion', 'feeling', 'vocal style', 'vocal range', 'harmonic structure',
        'rhythmic pattern', 'time signature', 'lyrical content', 'language'
    ]
    
    # Questions that are typically about single events
    single_event_keywords = [
        'instrument plays', 'which instrument', 'sound effect', 'vocal technique',
        'specific technique', 'at the beginning', 'at the end', 'interrupts',
        'distinctive characteristic', 'unique element', 'particular sound',
        'specific moment', 'brief appearance', 'sound at start', 'sound at end'
    ]
    
    # Check for long-term understanding keywords
    for keyword in long_term_keywords:
        if keyword in prompt_lower:
            return False
    
    # Check for single event keywords
    for keyword in single_event_keywords:
        if keyword in prompt_lower:
            return True
    
    # Default to False for ambiguous cases (requires longer understanding)
    return False

# Apply the function to each row
df['question'] = df.apply(lambda row: row['prompt'].split("Question: ")[-1], axis=1) # skip in-context example
df['single_sounding_event'] = df.apply(lambda row: is_single_sounding_event(row['question'], row['answers']), axis=1)

In [18]:
# after manual inspection, this is what we got
single_sounding_events = [337, 375, 396, 400, 425, 427, 428,482, 499, 514,655, 661, 662, 666, 719, 828, 833, 869,980,995,1028,1075,1137,1176,]

In [19]:
# dataset_path = "/media/gigibs/DD02EEEC68459F17/datasets"
# dataset_path = "/scratch/gv2167/datasets/"
dataset_path = "/home/gigibs/Documents/datasets"

In [20]:
qfs = pd.read_json("../data/output_data/qwen_fs.json")
qzs = pd.read_json("../data/output_data/qwen_zs.json")
mfs = pd.read_json("../data/output_data/mu_fs.json")
mzs = pd.read_json("../data/output_data/mu_zs.json")

qfs = utils.parse_df(qfs, experiment_name="qwen_fs")
qzs = utils.parse_df(qzs, experiment_name="qwen_zs")
mfs = utils.parse_df(mfs, experiment_name="mu_fs").fillna(0)
mzs = utils.parse_df(mzs, experiment_name="mu_zs").fillna(0)

/home/gigibs/Documents/research/2025_investigating_mmshap/src/experiments/mmshap.py:20: RuntimeWarning: invalid value encountered in scalar divide
  text_score = text_contrib / (text_contrib + audio_contrib)# + 1e-5)
/home/gigibs/Documents/research/2025_investigating_mmshap/src/experiments/mmshap.py:21: RuntimeWarning: invalid value encountered in scalar divide
  audio_score = audio_contrib / (text_contrib + audio_contrib)# + 1e-5)
/home/gigibs/Documents/research/2025_investigating_mmshap/src/experiments/mmshap.py:20: RuntimeWarning: invalid value encountered in scalar divide
  text_score = text_contrib / (text_contrib + audio_contrib)# + 1e-5)
/home/gigibs/Documents/research/2025_investigating_mmshap/src/experiments/mmshap.py:21: RuntimeWarning: invalid value encountered in scalar divide
  audio_score = audio_contrib / (text_contrib + audio_contrib)# + 1e-5)


In [21]:
def get_shapley_values(row):
    question_id =  row.name
    
    data = f"../{row['output_folder']}/{question_id}_info.npz"
    data = np.load(data)
    tokens = row["input_ids"]
    audio_tokens = np.where(tokens < 0)[-1]
    question_tokens = np.where(tokens >= 0)[-1]
    
    all_shapley_values = data["shapley_values"].squeeze(0).squeeze(0)
    audio_shapley_values = all_shapley_values[audio_tokens]
    question_shapley_values = all_shapley_values[question_tokens]

    return all_shapley_values, audio_shapley_values, question_shapley_values

def weight_important_features(audio, audio_shapley_values):
    """
    given an audio and its shapley values, apply the magnitude to the audio windows and 
    return the processed audio
    """
    # Calculate segment length
    total_length = len(audio)
    num_segments = len(audio_shapley_values)
    segment_length = total_length // num_segments
    
    processed_audio = np.zeros_like(audio)
    
    for i, magnitude in enumerate(audio_shapley_values):
        start_idx = i * segment_length
        end_idx = (i + 1) * segment_length
        
        # For the last segment, include any remaining samples
        if i == num_segments - 1:
            end_idx = total_length
        
        processed_audio[start_idx:end_idx] = audio[start_idx:end_idx] * magnitude

    return processed_audio

def save_qual_results(sample, img_format="pdf", gt_start=None, gt_end=None):
    if sample["experiment"].startswith("qwen"):
        sample_rate = 16000
    else:
        sample_rate = 24000

    x, fs = librosa.load(os.path.join(dataset_path, sample.audio_path), sr=sample_rate)
    all_shapley_values, audio_shapley_values, question_shapley_values = get_shapley_values(sample)

    
    output_folder = f"{sample['experiment']}_{sample.name}"
    os.makedirs(output_folder, exist_ok=True)
    print("saving data in ", output_folder)
    
    # generate plot per whole output
    plots.visualize_shapley_analysis(
        text_shapley_values=question_shapley_values,
        question_tokens=sample["input_tokens"],
        answer_tokens=sample["output_tokens"],
        audio_signal=x,
        audio_shapley_values=audio_shapley_values,
        sample_rate=sample_rate,
        gt_start=gt_start,
        gt_end=gt_end,
        idx=None,
        figsize=(12,7),
        colormap="binary",
        threshold=0.8, 
        save_folder=output_folder,
        show_image=False,
        output_format=img_format
    )

    positive_contrib = np.clip(audio_shapley_values, a_min=0, a_max=None).sum(axis=1)
    neg_contrib = np.clip(audio_shapley_values, a_min=None, a_max=0).sum(axis=1)
    agg_contrib = audio_shapley_values.sum(axis=1)
   
    pos_wav = weight_important_features(x, positive_contrib)
    neg_wav = weight_important_features(x, neg_contrib)
    # agg_wav =  weight_important_features(x, agg_contrib)
    sf.write(os.path.join(output_folder, "agg_positive.wav"),  pos_wav, sample_rate)
    sf.write(os.path.join(output_folder, "agg_negative.wav"), neg_wav, sample_rate)
    # sf.write(os.path.join(output_folder, "agg_abs.wav"), agg_wav, sample_rate)

    # generate plots per token
    for i, t in enumerate(sample["output_tokens"]):
        highlighted_text = plots.visualize_shapley_analysis(
                text_shapley_values=question_shapley_values,
                question_tokens=sample["input_tokens"],
                answer_tokens=sample["output_tokens"],
                audio_signal=x,
                audio_shapley_values=audio_shapley_values,
                sample_rate=sample_rate,
                gt_start=gt_start,
                gt_end=gt_end,
                idx=i,
                figsize=(12,7),
                colormap="binary",
                threshold=0.3, 
                save_folder=output_folder,
                show_image=False,
                output_format=img_format
            )

        positive_contrib = np.clip(audio_shapley_values[:, i], a_min=0, a_max=None)
        neg_contrib = np.clip(audio_shapley_values[:, i], a_min=None, a_max=0)
        agg_contrib = audio_shapley_values[:, i]
        pos_wav = weight_important_features(x, positive_contrib)
        neg_wav = weight_important_features(x, neg_contrib)
        # agg_wav =  weight_important_features(x, agg_contrib)
        
        sf.write(os.path.join(output_folder, f"{i}_{t}_positive.wav"), pos_wav, sample_rate)
        sf.write(os.path.join(output_folder, f"{i}_{t}_negative.wav"), neg_wav, sample_rate)
        # sf.write(os.path.join(output_folder, f"{i}_{t}_abs.wav"), agg_wav, sample_rate)

    with open(os.path.join(output_folder, f"{output_folder}.txt"), "w") as f:
        f.write(sample["model_output"])
        
    return

In [ ]:
sse = pd.read_csv("single_sounding_events.csv")

sse.set_index("question_id", inplace=True)

In [24]:
sse

,gt_start,gt_end
question_id,,
337,5,10
396,0,3
400,8,10
425,0,3
427,3,5
482,8,10
499,4,6
655,0,4
661,6,10


In [ ]:
# generate all figures w approx ground truth
for exp in [mzs, mfs, qzs, qfs]:
    for i in single_sounding_events:
        try:
            save_qual_results(exp.loc[i], gt_start=sse.loc[i][0], gt_end=sse.loc[i][1])
        except Exception as e:
            print("could not process track", i)

/tmp/ipykernel_30695/762651962.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  save_qual_results(exp.loc[i], gt_start=sse.loc[i][0], gt_end=sse.loc[i][1])


saving data in  mu_zs_337
text_max 42.45125325520833, text_min -30.605326334635418, text_median -0.47770182291666674
audio_max 34.761383056640625, audio_min -29.008656819661457, audio_median -0.27577718098958337
max_abs_value 42.45125325520833
text_max 3.5916341145833335, text_min -3.896484375, text_median -0.14420572916666666
audio_max 2.6204427083333335, audio_min -3.1939493815104165, audio_median 0.06380208333333333
max_abs_value 6.566324869791667
text_max 6.179158528645833, text_min -4.107401529947917, text_median -0.056315104166666664
audio_max 2.9174702962239585, audio_min -2.7297770182291665, audio_median 0.026204427083333332
max_abs_value 6.566324869791667
text_max 3.3898111979166665, text_min -3.0590006510416665, text_median -0.11966959635416667
audio_max 2.0861002604166665, audio_min -2.1305338541666665, audio_median 0.11124674479166667
max_abs_value 6.566324869791667
text_max 2.46337890625, text_min -2.57666015625, text_median 0.03173828125
audio_max 2.2698567708333335, audi

/tmp/ipykernel_30695/762651962.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  save_qual_results(exp.loc[i], gt_start=sse.loc[i][0], gt_end=sse.loc[i][1])


text_max 3.5120442708333335, text_min -3.6966145833333335, text_median 0.21923828125
audio_max 3.9954427083333335, audio_min -3.5787760416666665, audio_median 0.044270833333333336
max_abs_value 7.908668518066406
text_max 2.8070170084635415, text_min -3.1527328491210938, text_median 0.35231526692708337
audio_max 2.3931070963541665, audio_min -3.28680419921875, audio_median -0.1943359375
max_abs_value 7.908668518066406
text_max 2.3331705729166665, text_min -2.4481608072916665, text_median 0.18007405598958334
audio_max 2.6909993489583335, audio_min -3.5216471354166665, audio_median -0.0913848876953125
max_abs_value 7.908668518066406
text_max 3.205322265625, text_min -3.4685872395833335, text_median 0.3836212158203125
audio_max 2.9664713541666665, audio_min -4.01318359375, audio_median -0.10410563151041666
max_abs_value 7.908668518066406


/home/gigibs/Documents/research/2025_investigating_mmshap/notebooks/plots_improved.py:43: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=figsize)


...

saving data in  qwen_fs_661
text_max 4.989501953125, text_min -2.448394775390625, text_median 0.2327880859375
audio_max 2.896240234375, audio_min -2.95361328125, audio_median 0.02099609375
max_abs_value 4.989501953125
text_max 0.96875, text_min -1.28125, text_median 0.03125
audio_max 0.21875, audio_min -0.15625, audio_median 0.0
max_abs_value 4.5546875
text_max 2.2177734375, text_min -1.796875, text_median 0.0234375
audio_max 1.01171875, audio_min -0.5859375, audio_median 0.0
max_abs_value 4.5546875
text_max 3.2734375, text_min -0.6484375, text_median 0.0
audio_max 0.609375, audio_min -0.5703125, audio_median 0.0
max_abs_value 4.5546875
text_max 0.953125, text_min -1.091796875, text_median 0.010498046875
audio_max 0.361328125, audio_min -1.58154296875, audio_median 0.00048828125
max_abs_value 4.5546875
text_max 0.7109375, text_min -0.9921875, text_median 0.0
audio_max 0.390625, audio_min -0.375, audio_median 0.015625
max_abs_value 4.5546875
text_max 0.897705078125, text_min -1.61083984

/tmp/ipykernel_30695/762651962.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  save_qual_results(exp.loc[i], gt_start=sse.loc[i][0], gt_end=sse.loc[i][1])


text_max 0.90625, text_min -0.59375, text_median 0.0
audio_max 0.4375, audio_min -0.6875, audio_median 0.0
max_abs_value 4.0390625
text_max 1.5234375, text_min -0.6171875, text_median 0.0244140625
audio_max 0.625, audio_min -0.1484375, audio_median 0.0078125
max_abs_value 4.0390625
text_max 3.8095703125, text_min -1.5703125, text_median 0.01171875
audio_max 1.0703125, audio_min -0.833984375, audio_median 0.0126953125
max_abs_value 4.0390625
text_max 0.71875, text_min -1.54296875, text_median 0.0234375
audio_max 0.3828125, audio_min -0.34375, audio_median 0.01171875
max_abs_value 4.0390625
text_max 1.421875, text_min -1.490234375, text_median 0.02734375
audio_max 0.15234375, audio_min -0.446044921875, audio_median 0.013278961181640625
max_abs_value 4.0390625
text_max 1.46875, text_min -1.01171875, text_median 0.0078125
audio_max 0.6484375, audio_min -0.484375, audio_median 0.0078125
max_abs_value 4.0390625
text_max 1.26171875, text_min -0.271484375, text_median 0.03125
audio_max 0.32519

/tmp/ipykernel_30695/762651962.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  save_qual_results(exp.loc[i], gt_start=sse.loc[i][0], gt_end=sse.loc[i][1])


saving data in  qwen_fs_833
text_max 25.83050537109375, text_min -16.44873046875, text_median 0.32763671875
audio_max 8.627197265625, audio_min -20.4794921875, audio_median 0.030029296875
max_abs_value 25.83050537109375
